# Tensorflow X Keras: Buiding Blocks (How to Play)

It is a summary for Tensorflow building blocks, and guide to setup models in it.

The whole document is divided into 
1. 
2. 
3. 
4. 
...

## 1. Basic Procedures

Tensorflow model implementation is only two parts:
1. Building a computational graph
2. Running a computatioal graph

### Fundamental Element -- Tensor

A 3D tensor with shape $[a, b, c]$ can be understood as below
1. a is # of elements in 1st outer bracket (how many brackets in the outermost bracket)
2. b is # of elements in 2nd outer bracket
3. c is # of elements in 3rd outer bracket

<img src="img/tensor.png" alt="title" >


### Build Computational Graph

Graph is consisted by nodes of four kinds:
- **Constant:** *NO* input --> output internally stored values
    - `tf.constant(value, dtype, shape)`
    - initialized when call `tf.constant`
- **Operation:** one or more inputs --> output operation results

<img src="img/node1.png" alt="title" >

The most easy way of starting a session is:
1. create a instance `sess = tf.Session()`
2. then evaluate any nodes with `sess.run(<nodes you want to evaluate>)`

The other option better for IPython notebook is to use `sess = tf.InteractiveSession()`. This avoids having to pass an explicit Session object to run ops.

The reason why using session is that:
> Instead of running a single expensive operation independently from Python, which has high cost to transferring data, TensorFlow lets us describe a graph of interacting operations that run entirely outside Python.

In [19]:
### Example of constant and operation nodes
import tensorflow as tf

# Create seperated constant nodes in computational graph
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0)  # Also tf.float32 implicitly
print(node1, node2, "\n")  # This only outputs the chracteristics of these two nodes, without evaluate it

# To evaluate the node and output the result
sess = tf.Session()
print(sess.run([node1, node2]), "\n")

# Add in a operation nodes of an operation
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32) 

[3.0, 4.0] 

node3: Tensor("Add_1:0", shape=(), dtype=float32)
sess.run(node3): 7.0


In [ ]:
### Example of tf.InteractiveSession()
sess = tf.InteractiveSession()
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b

# We can just use 'c.eval()' without passing 'sess'
print(c.eval())
sess.close()

### Regular session example
with tf.Session():
  # We can also use 'c.eval()' here.
  print(c.eval())

-------
The other two kinds of nodes are:
- **Placeholder:** *NO* input --> output external feed values
    - `tf.placeholder(dtype, shape)`
    - feed data when call 'sess.run'
    - <img src="img/placeholder.png" alt="title" >
- **Variable:** *NO* input --> output trainable parameters
    - `tf.get_variable` (high API) and `tf.Variable(<initial-value>)` (low API) 
    - initialized when call `init = tf.global_variables_initializer()` and `sess.run(init)`

In [20]:
### Example of placeholder and variable nodes
# Variables
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)

# Input and Output
x = tf.placeholder(tf.float32)
linear_model = W*x + b
y = tf.placeholder(tf.float32)

# Loss function
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)

# Initialize variable (reset variables)
init = tf.global_variables_initializer()
sess.run(init)  # After this step, can also assign specific value to variable with tf.assign(<variable name>, [<value>]) 

print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


### Train Model

Tensorflow will automatically handles derivatives after setup optimizer.

In [21]:
### Create optimizer after loss function is specified
optimizer = tf.train.GradientDescentOptimizer(0.01)  # Types of optimizer
train = optimizer.minimize(loss)  # What is going to be optimized

In [22]:
### Start training
sess.run(init) # Reset values to incorrect defaults

for i in range(1000):
  sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


<img src="img/linear.png" alt="title" >

## 2. Build and Train Model via tf.estimator

Higher API for tensorflow model building and training. [Official Introduction](https://www.tensorflow.org/get_started/get_started)

## 3. TensorBoard

Write a log file consisting model training details (Event protocol buffer) and open it with TensorBoard to visualize it.

By using `add` method of `tf.summary.FileWriter`, you can specify what summaries you need to collect. For example, you are going to visualize the graph before training:

``` python
sess.run(tf.global_variables_initializer())

writer = tf.summary.FileWriter(logdir)
writer.add_graph(sess.graph)
```

Except for the graph, normally we collect all summaries periodically when training the model.

In [ ]:
### Well specified groups of neural network components
### helps meaningful visualization

In [ ]:
### The simplest way is collecting all summaries of the graph
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter(logdir)
writer.add_graph(sess.graph)

for i in range(2001):
    batch = mnist.train.next_batch(100)
    if i % 5 == 0:
        s = sess.run(merged_summary, feed_dict={x: batch[0], y: batch[1]})
        writer.add_summary(s, i)
    sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})
    
### Run Tensorboard via common line
>> tensorboard --logdir <full path of log file>